In [1]:
import keras

In [2]:
dataset_path = 'dataset'

train_path = dataset_path + '/train'
train_images_path = train_path + '/images'
train_annotations_path = train_path + '/annotations'

valid_path = dataset_path + '/valid'
valid_images_path = valid_path + '/images'
valid_annotations_path = valid_path + '/annotations'

test_path = dataset_path + '/test'
test_images_path = test_path + '/images'
test_annotations_path = test_path + '/annotations'

In [3]:
# images are .png images
# annotations are .json files
# and have the following format:
"""
{
    "version": "5.4.1",
    "flags": {},
    "shapes": [
        {
            "label": "femurR",
            "points": [
                [
                    598.125,
                    892.5
                ]
            ],
            "group_id": null,
            "description": "",
            "shape_type": "point",
            "flags": {},
            "mask": null
        },
        {
            "label": "femurR",
            "points": [
                [
                    658.125,
                    903.125
                ]
            ],
            "group_id": null,
            "description": "",
            "shape_type": "point",
            "flags": {},
            "mask": null
        },
        {
            "label": "femurR",
            "points": [
                [
                    670.625,
                    983.75
                ]
            ],
            "group_id": null,
            "description": "",
            "shape_type": "point",
            "flags": {},
            "mask": null
        },
        {
            "label": "femurL",
            "points": [
                [
                    1170.0,
                    892.5
                ]
            ],
            "group_id": null,
            "description": null,
            "shape_type": "point",
            "flags": {},
            "mask": null
        },
        {
            "label": "femurL",
            "points": [
                [
                    1115.0,
                    894.375
                ]
            ],
            "group_id": null,
            "description": "",
            "shape_type": "point",
            "flags": {},
            "mask": null
        },
        {
            "label": "femurL",
            "points": [
                [
                    1086.25,
                    969.375
                ]
            ],
            "group_id": null,
            "description": null,
            "shape_type": "point",
            "flags": {},
            "mask": null
        },
        {
            "label": "acetabulumR",
            "points": [
                [
                    611.25,
                    871.25
                ]
            ],
            "group_id": null,
            "description": "",
            "shape_type": "point",
            "flags": {},
            "mask": null
        },
        {
            "label": "acetabulumL",
            "points": [
                [
                    1148.75,
                    858.75
                ]
            ],
            "group_id": null,
            "description": "",
            "shape_type": "point",
            "flags": {},
            "mask": null
        }
    ],
    "imageHeight": 2140,
    "imageWidth": 1760
}
"""
# the model should be able to receive the image and predict the points for all the labels

'\n{\n    "version": "5.4.1",\n    "flags": {},\n    "shapes": [\n        {\n            "label": "femurR",\n            "points": [\n                [\n                    598.125,\n                    892.5\n                ]\n            ],\n            "group_id": null,\n            "description": "",\n            "shape_type": "point",\n            "flags": {},\n            "mask": null\n        },\n        {\n            "label": "femurR",\n            "points": [\n                [\n                    658.125,\n                    903.125\n                ]\n            ],\n            "group_id": null,\n            "description": "",\n            "shape_type": "point",\n            "flags": {},\n            "mask": null\n        },\n        {\n            "label": "femurR",\n            "points": [\n                [\n                    670.625,\n                    983.75\n                ]\n            ],\n            "group_id": null,\n            "description": "",\n     

In [4]:
# were we put the valid images in the train directory
# since we are doing the split during the training with validation_split = 0.2

import shutil
import os

# Define the source and destination directories
valid_images_path = os.path.join(dataset_path, 'valid', 'images')
train_images_path = os.path.join(dataset_path, 'train', 'images')

# Get a list of all the image files in the valid directory
valid_image_files = os.listdir(valid_images_path)

# Move each file to the train directory
for file_name in valid_image_files:
    shutil.move(os.path.join(valid_images_path, file_name), train_images_path)

In [5]:
import os
import json
import cv2
import numpy as np
from glob import glob
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define the splits
splits = ['train', 'valid', 'test']

# Function to sort points
def sort_points(annotation):
    # Define the order of labels
    label_order = ['femurR', 'femurL', 'acetabulumR', 'acetabulumL']

    # Create a list of tuples, each containing a point and its corresponding label
    points_with_labels = [(point, shape['label']) for shape in annotation['shapes'] for point in shape['points']]
    # Sort the points based on the predefined label order
    points_with_labels.sort(key=lambda x: label_order.index(x[1]))
    # Remove the labels, leaving only the sorted points
    sorted_points = [point for point, label in points_with_labels]

    return sorted_points

# Function to load images and annotations
def load_data(images_path, annotations_path):
    # Get a list of all image files and annotation files
    image_files = sorted(glob(os.path.join(images_path, '*.png')))
    annotation_files = sorted(glob(os.path.join(annotations_path, '*.json')))

    #print(f'Found {len(image_files)} image files and {len(annotation_files)} annotation files.')

    images = []
    annotations = []
    image_paths = []

    for image_file, annotation_file in zip(image_files, annotation_files):
        # Load and resize the image
        image = load_img(image_file, target_size=(64, 64))
        image = img_to_array(image)
        images.append(image)

        # Load and process the annotation
        with open(annotation_file, 'r') as f:
            annotation = json.load(f)
        points = sort_points(annotation)
        annotations.append(points)

        # Save the image path
        image_paths.append(image_file)

    return np.array(images), np.array(annotations), image_paths

# Load data for each split
for split in splits:
    images_path = os.path.join(dataset_path, split, 'images')
    annotations_path = os.path.join(dataset_path, split, 'annotations')
    split_images, split_annotations, split_image_paths = load_data(images_path, annotations_path)

    if split == 'train':
        train_images, train_annotations, train_image_paths = split_images, split_annotations, split_image_paths
    elif split == 'valid':
        valid_images, valid_annotations, valid_image_paths = split_images, split_annotations, split_image_paths
    elif split == 'test':
        test_images, test_annotations, test_image_paths = split_images, split_annotations, split_image_paths

    print(f'Loaded {len(split_images)} images and {len(split_annotations)} annotations for {split}.')

Loaded 115 images and 115 annotations for train.
Loaded 0 images and 0 annotations for valid.
Loaded 17 images and 17 annotations for test.


In [6]:
# here we create a model that will be used to train
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Determine the maximum number of points in any annotation
num_points = max(len(annotation) for annotation in train_annotations)

# Create a sequential model
model = Sequential()

# Add a convolutional layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))

# Add a max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add another convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))

# Add another max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the tensor output from the previous layer
model.add(Flatten())

# Add a dense layer
model.add(Dense(128, activation='relu'))

# Add the output layer
model.add(Dense(num_points * 2))

# Define the optimizer with a specific learning rate
optimizer = Adam(learning_rate=0.001)

# Compile the model
model.compile(optimizer=optimizer, loss='mse')

# Print the model summary
model.summary()

/opt/homebrew/Caskroom/miniconda/base/envs/keypoint_detection/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │         2,064 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,627,216 (6.21 MB)

 Trainable params: 1,627,216 (6.21 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Just to check how the model is receving all the data

# Convert the lists to numpy arrays
train_images = np.array(train_images)
train_annotations = np.array(train_annotations)

# Print the first image and annotation before processing
print("Before processing:")
print("First image path:", train_image_paths[0])
print("First image shape:", train_images[0].shape)
print("First image data:", train_images[0])
print("First annotation shape:", train_annotations[0].shape)
print("First annotation data:", train_annotations[0])

# Reshape the annotations and normalize the images
train_annotations = train_annotations.reshape(len(train_annotations), -1)
train_images = train_images / 255.0

# Print the first image and annotation after processing
print("\nAfter processing:")
print("First image path:", train_image_paths[0])
print("First image shape:", train_images[0].shape)
print("First image data:", train_images[0])
print("First annotation shape:", train_annotations[0].shape)
print("First annotation data:", train_annotations[0])

Before processing:
First image path: dataset/train/images/0001_1_PF.png
First image shape: (64, 64, 3)
First image data: [[[ 5.  5.  5.]
  [ 4.  4.  4.]
  [ 3.  3.  3.]
  ...
  [ 3.  3.  3.]
  [ 3.  3.  3.]
  [ 3.  3.  3.]]

 [[ 4.  4.  4.]
  [ 4.  4.  4.]
  [ 3.  3.  3.]
  ...
  [ 3.  3.  3.]
  [ 3.  3.  3.]
  [ 4.  4.  4.]]

 [[ 4.  4.  4.]
  [ 4.  4.  4.]
  [ 3.  3.  3.]
  ...
  [ 3.  3.  3.]
  [ 3.  3.  3.]
  [ 4.  4.  4.]]

 ...

 [[ 3.  3.  3.]
  [ 2.  2.  2.]
  [ 0.  0.  0.]
  ...
  [ 2.  2.  2.]
  [ 2.  2.  2.]
  [ 3.  3.  3.]]

 [[ 2.  2.  2.]
  [ 2.  2.  2.]
  [ 0.  0.  0.]
  ...
  [ 0.  0.  0.]
  [ 2.  2.  2.]
  [ 3.  3.  3.]]

 [[46. 46. 46.]
  [41. 41. 41.]
  [36. 36. 36.]
  ...
  [ 2.  2.  2.]
  [ 2.  2.  2.]
  [ 3.  3.  3.]]]
First annotation shape: (8, 2)
First annotation data: [[ 598.125  892.5  ]
 [ 658.125  903.125]
 [ 670.625  983.75 ]
 [1170.     892.5  ]
 [1115.     894.375]
 [1086.25   969.375]
 [ 611.25   871.25 ]
 [1148.75   858.75 ]]

After processing:
First i

In [8]:
# Convert the lists to numpy arrays
train_images = np.array(train_images)
train_annotations = np.array(train_annotations).reshape(len(train_annotations), -1)

# Normalize the images
train_images = train_images / 255.0

# Train the model
print(f'Training on {len(train_images)} images')
model.fit(train_images, train_annotations, epochs=10, validation_split=0.2, batch_size=32)

Training on 115 images
Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 1559492.6250 - val_loss: 1142737.0000
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 1617612.5000 - val_loss: 1142148.3750
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 1562964.6250 - val_loss: 1140610.7500
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 1538838.0000 - val_loss: 1137338.7500
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 1605098.8750 - val_loss: 1131210.7500
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 1551766.5000 - val_loss: 1120606.6250
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 1582917.7500 - val_loss: 1103363.5000
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 1505291.1250 - val_loss: 1076607.6250
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 1510593.5000 - val_loss: 1036890.9375
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 1386946.3750 - val_loss: 980147.7500
